In [66]:
import sklearn.metrics
import torch.nn as nn
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import importlib
import src.params as params
import src.model as model
import src.main as main
import src.dataloader as dataloader
importlib.reload(dataloader)
importlib.reload(model)
import torch.nn.functional as F
import src.dataloader as dataloader
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler
import src.preprocess as preprocess
from copy import copy
importlib.reload(preprocess)
import src.feature_extraction as feature_extraction
importlib.reload(feature_extraction)
import sys, os, torch

In [2]:
args = params.get_args("../config/system_a.yaml")

In [8]:
a = np.array([[np.nan, 1, 2, 2, 1], [2,np.nan,2,3,3]])
a = a.transpose()
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit_transform(a)
a = np.random.random((2,3))
np.pad(a, ((0,0), (1,2)))
a = np.array([[1,2,3],[5,6,7]])
print(a.shape)
a.flatten(order='F')

In [4]:
gt = dataloader.load_preprocessed(os.path.join(args.workdir, "gt_pre.pkl"))
gt = preprocess.preprocess_gt(gt, args.start_time)

In [5]:
cmdb, cmdb_kpi = main.get_cmdb(args)

Loading preprocessed data from /home/junetheriver/code/aiops/workdir/system_a/data_pre.pkl


In [93]:
ts_a, ts_b, y_df = feature_extraction.feature_extraction(cmdb, cmdb_kpi, gt, args.window, args)

In [25]:
data_size = y_df.size

In [98]:
ft_a = DataFrame(columns=ts_a.columns, index=ts_a.index, data=None)
ft_a.id = ts_a.id
ft_a.timestamp = ts_a.timestamp

,id,timestamp,system.os.nofile.current,system.tcp.close_wait,system.tcp.fin_wait1,system.fs.inodes.used,system.net.tcp.out_segs,system.net.bytes_rcvd,system.load.norm.1,system.net.bytes_sent,...,redis.replication.master_repl_offset,redis.mem.used,system.io.rrqm_s,haproxy.frontend.errors.req_rate,system.fs.inodes.in_use,weblogic.jdbc.active_conn_avg,haproxy.frontend.session.pct,haproxy.backend_up,weblogic.cluster.status,system.tcp.retrans_pct
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,388,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,388,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,388,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,388,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
for i in range(data_size):
    # print(ts_a[ts_a.id == i])
    ts = ts_a[ts_a.id == i].loc[:,ts_a.columns[2]:]
    ft = abs(np.fft.fft(ts, axis=0))
    # print(i, ft_a[ft_a.id==i].shape, ft.shape)
    ft_a.loc[ft_a.id == i,ts_a.columns[2]:] = ft

In [8]:
ts_a, ts_b, ft_a, ft_b, y_df = feature_extraction.feature_extraction(cmdb, cmdb_kpi, gt, args.window, args)

In [35]:
data_size = ts_a.id.max()+1
describe = []
for i in range(data_size):
    ts = ts_a[ts_a.id == i].loc[:, ts_a.columns[2]:]
    describe.append(ts.describe()[1:].values.flatten(order='F'))
describe = np.stack(describe)


In [67]:
pp = feature_extraction.feature_extraction(cmdb, cmdb_kpi, gt, args.window, args)

In [70]:
np.isnan(pp["after_stat"][:,-1]).all()

False

In [89]:
pp["before_stat"][0,:].shape

(651,)

In [77]:
import pickle
with open("../../workdir/system_a/preprocess_data.pkl", 'wb') as f:
    pickle.dump(pp, f)

In [102]:
from torch.utils.data import Dataset, DataLoader
class MyDataset2(Dataset):
    def __init__(self, pp):
        self.pp = pp
    def __len__(self):
        return self.pp["y"].size
    def __getitem__(self, item):
        dt = {
            "before": pp["before"][pp["before"].id == item].values[:,2:],
            "after": pp["after"][pp["after"].id == item].values[:,2:],
            "before_fft": pp["before_fft"][pp["before_fft"].id == item].values[:,2:],
            "after_fft": pp["after_fft"][pp["after_fft"].id == item].values[:,2:],
            "before_stat": pp["before_stat"][item,:],
            "after_stat": pp["after_stat"][item,:],
            "before_fft_stat": pp["before_fft_stat"][item,:],
            "after_fft_stat": pp["after_fft_stat"][item,:],
        }
        return dt, self.pp["y"][item]

In [103]:
dataset = MyDataset2(pp)

In [105]:
for i in dataset:
    print(i)
    break

({'before': array([[ 0.63165848, -0.20957167,  1.09991568,  1.10260239, -0.29135013,
        -0.22674713,  0.67944491,  0.6446207 , -1.179962  ,  1.04118661,
        -1.17986202,  1.02684198, -0.33743395,  1.07412727, -0.44215458,
         1.17987451,  0.46716419,  0.32953601, -0.30791229, -0.50394517,
         1.23639483, -0.17389697,  2.29516287,  0.44215458,  2.92164692,
        -0.67398875, -0.64626385,  1.09434286, -0.3795776 ,  0.90443881,
         1.20414457, -0.67944491,  1.20481394,  1.5958171 ,  1.00343548,
        -0.46716419,  1.00118117,  0.63068778,  0.83244017,  0.90033994,
         0.87278916,  1.04664182,  0.9104649 , -1.59453366, -0.39177431,
        -0.55031223,  1.67844056, -0.36497535, -0.07404737,  0.77736898,
         1.04214283,  1.08088141, -0.18198463,  0.69152661,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,      